## 生成mask

In [ ]:
import os
import cv2
import numpy as np

def process_image(image_path, output_folder):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((5, 5), np.uint8)
    binary_cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    binary_cleaned = cv2.morphologyEx(binary_cleaned, cv2.MORPH_OPEN, kernel)
    contours, _ = cv2.findContours(binary_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    result_mask = np.zeros_like(gray)
    min_area = np.pi * (12 ** 2)
    max_area = np.pi * (20 ** 2)

    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            continue
        circularity = 4 * np.pi * (area / (perimeter ** 2))
        if min_area < area < max_area and 0.8 < circularity < 1.2:
            cv2.drawContours(result_mask, [contour], -1, 255, -1)

    colored_mask = cv2.merge([result_mask, result_mask, result_mask])
    colored_mask[:, :, 1:] = 0
    alpha = 0.5
    overlay = cv2.addWeighted(colored_mask.astype(np.uint8), alpha, image, 1 - alpha, 0)

    base_name = os.path.splitext(os.path.basename(image_path))[0]
    mask_output_path = os.path.join(output_folder, base_name + ".jpg")
    overlay_output_path = os.path.join(output_folder, base_name + "_overlay.jpg")

    cv2.imwrite(mask_output_path, result_mask)
    cv2.imwrite(overlay_output_path, overlay)
    print ('save mask to {mask_output_path}')

def process_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            process_image(image_path, output_folder)

input_folder = "C:/Users/Vivo/CGU/odep_cellarray/Testimage/2024.11.29"
mask_folder = "C:/Users/Vivo/CGU/odep_cellarray/Testimage/mask_11.29"
radius_reduction = 6

process_folder(input_folder, mask_folder)


# mask_back to json (labelme)

In [3]:
import json
import base64
import cv2
import os

def image_to_base64(image_path):
    """将图像文件转换为 Base64 编码的字符串。"""
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

def mask_to_annotations(mask_path, radius_reduction):
    """从掩膜图像生成标注数据，过滤小于半径条件的圆。"""
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    if mask is None:
        raise ValueError(f"无法读取掩膜图像: {mask_path}")
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    annotations = []

    for contour in contours:
        (x, y), radius = cv2.minEnclosingCircle(contour)
        center = (int(x), int(y))
        radius = int(radius)

        # 过滤掉半径小于5的圆
        if radius < 5:
            continue
        
        # 生成圆形标注的点（圆心和缩小后的圆周上的一点）
        points = [
            [center[0], center[1]],  # 圆心
            [center[0] + (radius - radius_reduction), center[1]]  # 圆周上的一点（缩小后的半径）
        ]

        annotations.append({
            "label": "target",
            "points": points,
            "group_id": None,
            "description": "",
            "shape_type": "circle",
            "flags": {},
            "mask": None  # mask 初始为 None
        })
    return annotations

def create_labelme_json(original_image_path, mask_path, radius_reduction):
    """生成包含原始图像和掩膜的 JSON 文件。"""
    annotations = mask_to_annotations(mask_path, radius_reduction)

    json_data = {
        "version": "5.5.0",
        "flags": {},
        "shapes": annotations,
        "imagePath": os.path.basename(original_image_path),
        "imageData": image_to_base64(original_image_path),
        "imageHeight": 2076,  # 根据实际图像高度修改
        "imageWidth": 3088    # 根据实际图像宽度修改
    }

    return json_data

def process_folder(input_folder, mask_folder, radius_reduction):
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            original_image_path = os.path.join(input_folder, filename)
            mask_path = os.path.join(mask_folder, filename)
            if not os.path.exists(mask_path):
                print(f"Mask not found for {original_image_path}")
                continue

            json_output = create_labelme_json(original_image_path, mask_path, radius_reduction)
            json_file_name = os.path.splitext(filename)[0] + ".json"
            json_file_path = os.path.join(input_folder, json_file_name)

            with open(json_file_path, "w") as json_file:
                json.dump(json_output, json_file, indent=4)

            print(f"生成的 JSON 文件已保存為 {json_file_path}")

input_folder = "C:/Users/Vivo/CGU/odep_cellarray/Testimage/2024.11.29"
mask_folder = "C:/Users/Vivo/CGU/odep_cellarray/Testimage/mask_11.29"
radius_reduction = 6  # 調整

process_folder(input_folder, mask_folder, radius_reduction)

生成的 JSON 文件已保存為 C:/Users/Vivo/CGU/odep_cellarray/Testimage/2024.11.29\20241129_134.json
生成的 JSON 文件已保存為 C:/Users/Vivo/CGU/odep_cellarray/Testimage/2024.11.29\20241129_136.json
生成的 JSON 文件已保存為 C:/Users/Vivo/CGU/odep_cellarray/Testimage/2024.11.29\20241129_150.json
生成的 JSON 文件已保存為 C:/Users/Vivo/CGU/odep_cellarray/Testimage/2024.11.29\20241129_166.json
生成的 JSON 文件已保存為 C:/Users/Vivo/CGU/odep_cellarray/Testimage/2024.11.29\20241129_191.json
生成的 JSON 文件已保存為 C:/Users/Vivo/CGU/odep_cellarray/Testimage/2024.11.29\20241129_192.json
生成的 JSON 文件已保存為 C:/Users/Vivo/CGU/odep_cellarray/Testimage/2024.11.29\20241129_193.json
生成的 JSON 文件已保存為 C:/Users/Vivo/CGU/odep_cellarray/Testimage/2024.11.29\20241129_194.json
生成的 JSON 文件已保存為 C:/Users/Vivo/CGU/odep_cellarray/Testimage/2024.11.29\20241129_195.json
生成的 JSON 文件已保存為 C:/Users/Vivo/CGU/odep_cellarray/Testimage/2024.11.29\20241129_196.json
生成的 JSON 文件已保存為 C:/Users/Vivo/CGU/odep_cellarray/Testimage/2024.11.29\20241129_197.json
生成的 JSON 文件已保存為 C:/Users/Vivo/CG